1. Create the Prediciton Probability "Error" results for the xgboost and ffnn models analagously to the transformer model produced above.

In [6]:
import numpy as np
import pandas as pd
df_indicators = pd.read_csv('country_indicators.csv')
df_preds = pd.read_csv('test_predictions.csv')

In [9]:
#QUESTION1
df_preds['xgboost_probability_prediction_error'] = np.abs(df_preds['y_true_xgboost'].astype(float) - df_preds['y_pred_proba_xgboost'])
df_preds[['y_true_xgboost','y_pred_proba_xgboost','xgboost_probability_prediction_error']]

,y_true_xgboost,y_pred_proba_xgboost,xgboost_probability_prediction_error
0,False,0.066500,0.066500
1,False,0.099643,0.099643
2,True,0.704086,0.295914
3,True,0.638444,0.361556
4,False,0.608380,0.608380
...,...,...,...
359,False,0.079453,0.079453
360,False,0.060189,0.060189
361,True,0.697625,0.302375
362,False,0.729246,0.729246


In [10]:
df_preds['ffnn_probability_prediction_error'] = np.abs(df_preds['y_true_ffnn'].astype(float) - df_preds['y_pred_proba_ffnn'])
df_preds[['y_true_ffnn','y_pred_proba_ffnn','ffnn_probability_prediction_error']]

,y_true_ffnn,y_pred_proba_ffnn,ffnn_probability_prediction_error
0,False,0.409958,0.409958
1,False,0.406696,0.406696
2,False,0.545236,0.545236
3,False,0.534560,0.534560
4,True,0.538583,0.461417
...,...,...,...
359,False,0.291874,0.291874
360,False,0.300321,0.300321
361,False,0.335496,0.335496
362,False,0.324000,0.324000


2. Create a bootsrap confidence interval for the average Prediction Probability "Error" for one of these models using all the data.


> In this context "using all the data" means using all the predictions made for a given model under consideration; whereas, "using a data subset" would mean restricting the rows of the data on the basis of one (or more) of the columns from the "Progress Indicators" data so as to only consider the predictions made for the given model under consideration within a given subset of (the countries of) the data.

In [16]:
n = 5
repititions = 5000
transformer_stat = np.zeros(repititions)
sample_1 = np.arange(n)

# bootstrapping a single sample
for i in range(repititions):
    bootstrap_sample_transformer = np.random.choice(sample_1, size=n, replace = True)
    transformer_stat[i] = bootstrap_sample_transformer.mean()

np.quantile(transformer_stat,[0.025, 0.975])

array([0.8, 3.2])

3. Create the Prediction Classification "Correctness" results of "correct" and "incorrect" predictions for the `transformer`, `xgboost` and `ffnn` models; or, an alternative "either/or" breakdown of interest (such as "wrongly predicted no escalation" versus all the other categories combined).

In [11]:
threshold = 0.5 

df_preds['transformer_classifcation_performance_outcome'] = None

tmp = df_preds['transformer_classifcation_performance_outcome'].copy()
TP_pos_pred_correct = df_preds.y_true_transformer & (df_preds.y_pred_proba_transformer>threshold)
tmp[TP_pos_pred_correct] = "correctly predicted escalation"
TN_neg_pred_correct = (~df_preds.y_true_transformer) & (df_preds.y_pred_proba_transformer<=threshold)
tmp[TN_neg_pred_correct] = "correctly predicted no escalation"
FP_pos_pred_wrong = (~df_preds.y_true_transformer) & (df_preds.y_pred_proba_transformer>threshold)
tmp[FP_pos_pred_wrong] = "wrongly predicted escalation"
FN_neg_pred_wrong = df_preds.y_true_transformer & (df_preds.y_pred_proba_transformer<=threshold)
tmp[FN_neg_pred_wrong] = "wrongly predicted no escalation"

df_preds['transformer_classifcation_performance_outcome'] = tmp
df_preds[['y_true_transformer','y_pred_transformer','transformer_classifcation_performance_outcome']][30:50]

,y_true_transformer,y_pred_transformer,transformer_classifcation_performance_outcome
30,True,True,correctly predicted escalation
31,False,True,wrongly predicted escalation
32,False,True,wrongly predicted escalation
33,False,True,wrongly predicted escalation
34,False,False,correctly predicted no escalation
35,False,True,wrongly predicted escalation
36,False,False,correctly predicted no escalation
37,False,False,correctly predicted no escalation
38,False,True,wrongly predicted escalation
39,False,True,wrongly predicted escalation


In [12]:
threshold = 0.5 

df_preds['xgboost_classifcation_performance_outcome'] = None

xmp = df_preds['xgboost_classifcation_performance_outcome'].copy()
xP_pos_pred_correct = df_preds.y_true_xgboost & (df_preds.y_pred_proba_xgboost>threshold)
xmp[xP_pos_pred_correct] = "correctly predicted escalation"
xN_neg_pred_correct = (~df_preds.y_true_xgboost) & (df_preds.y_pred_proba_xgboost<=threshold)
xmp[xN_neg_pred_correct] = "correctly predicted no escalation"
FP_pos_pred_wrong = (~df_preds.y_true_xgboost) & (df_preds.y_pred_proba_xgboost>threshold)
xmp[FP_pos_pred_wrong] = "wrongly predicted escalation"
FN_neg_pred_wrong = df_preds.y_true_xgboost & (df_preds.y_pred_proba_xgboost<=threshold)
xmp[FN_neg_pred_wrong] = "wrongly predicted no escalation"

df_preds['xgboost_classifcation_performance_outcome'] = xmp
df_preds[['y_true_xgboost','y_pred_xgboost','xgboost_classifcation_performance_outcome']][30:50]

,y_true_xgboost,y_pred_xgboost,xgboost_classifcation_performance_outcome
30,False,True,wrongly predicted escalation
31,False,True,wrongly predicted escalation
32,False,True,wrongly predicted escalation
33,False,True,wrongly predicted escalation
34,False,False,correctly predicted no escalation
35,False,True,wrongly predicted escalation
36,False,True,wrongly predicted escalation
37,False,True,wrongly predicted escalation
38,True,True,correctly predicted escalation
39,False,True,wrongly predicted escalation


In [13]:
threshold = 0.5 

df_preds['ffnn_classifcation_performance_outcome'] = None

fmp = df_preds['ffnn_classifcation_performance_outcome'].copy()
fP_pos_pred_correct = df_preds.y_true_ffnn & (df_preds.y_pred_proba_ffnn>threshold)
fmp[fP_pos_pred_correct] = "correctly predicted escalation"
fN_neg_pred_correct = (~df_preds.y_true_ffnn) & (df_preds.y_pred_proba_ffnn<=threshold)
fmp[fN_neg_pred_correct] = "correctly predicted no escalation"
FP_pos_pred_wrong = (~df_preds.y_true_ffnn) & (df_preds.y_pred_proba_ffnn>threshold)
fmp[FP_pos_pred_wrong] = "wrongly predicted escalation"
FN_neg_pred_wrong = df_preds.y_true_ffnn & (df_preds.y_pred_proba_ffnn<=threshold)
fmp[FN_neg_pred_wrong] = "wrongly predicted no escalation"

df_preds['ffnn_classifcation_performance_outcome'] = fmp
df_preds[['y_true_ffnn','y_pred_ffnn','ffnn_classifcation_performance_outcome']][30:50]

,y_true_ffnn,y_pred_ffnn,ffnn_classifcation_performance_outcome
30,True,True,correctly predicted escalation
31,False,False,correctly predicted no escalation
32,False,True,wrongly predicted escalation
33,False,True,wrongly predicted escalation
34,False,True,wrongly predicted escalation
35,False,True,wrongly predicted escalation
36,False,True,wrongly predicted escalation
37,False,False,correctly predicted no escalation
38,False,False,correctly predicted no escalation
39,False,False,correctly predicted no escalation
